In [1]:
from product.stock_category import StockCategory
from database.market import Market
from database.sec import SEC
from datetime import datetime, timedelta
from processor.processor import Processor as p
import pytz
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
factors = ['AccumulatedOtherComprehensiveIncomeLossNetOfTax',
 'Assets',
 'AssetsCurrent',
 'CashAndCashEquivalentsAtCarryingValue',
 'EarningsPerShareBasic',
 'EarningsPerShareDiluted',
 'EntityCommonStockSharesOutstanding',
 'Goodwill',
 'IncomeTaxExpenseBenefit',
 'LiabilitiesAndStockholdersEquity',
 'LiabilitiesCurrent',
 'OtherAssetsNoncurrent',
 'PropertyPlantAndEquipmentNet',
 'RetainedEarningsAccumulatedDeficit',
 'filed',
 'StockholdersEquity',
 'NetIncomeLoss']

In [3]:
market = Market()
sec = SEC()

## Currently not modeling stock category due to low actual accuracy

In [4]:
params = {}
product = StockCategory(params)
product.subscribe()
# sim = product.create_sim()
# product.load()
# # product.transformed

In [5]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [9]:
centralized = ['assets',
 'liabilitiesandstockholdersequity',
 'netcashprovidedbyusedinfinancingactivities',
 'netcashprovidedbyusedininvestingactivities',"date","ticker"]

In [7]:
sec.connect()
market.connect()
ticker = "MMM"
cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
filing = sec.retrieve_filing_data(cik)
filing["ticker"] = ticker
filing["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in filing["filed"]]
filing = p.column_date_processing(filing)
centralized = [x[:-1] if x[-1] == "s" else x for x in list(filing.columns)]
for ticker in tqdm(sp5["Symbol"]):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filing = sec.retrieve_filing_data(cik)
        filing["ticker"] = ticker
        filing["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in filing["filed"]]
        filing = p.column_date_processing(filing)
        current_columns = [x[:-1] if x[-1] == "s" else x for x in list(filing.columns)]
        if filing.index.size > 40:
            initial = [x for x in current_columns if x in centralized]
            centralized = [x for x in centralized if x in initial]
    except Exception as e:
        print(ticker, str(e))
sec.disconnect()
market.disconnect()

 39%|█████████████████████████████▋                                              | 197/505 [01:04<02:18,  2.22it/s]

FRC 'filed'


 83%|██████████████████████████████████████████████████████████████▊             | 417/505 [02:13<00:36,  2.40it/s]

SBNY 'filed'


100%|████████████████████████████████████████████████████████████████████████████| 505/505 [02:40<00:00,  3.15it/s]


In [10]:
filing

,adsh,accountspayablecurrent,accountsreceivablenetcurrent,accruedincometaxesnoncurrent,accumulateddepreciationdepletionandamortizationpropertyplantandequipment,accumulatedothercomprehensiveincomelossnetoftax,additionalpaidincapital,adjustmentstoadditionalpaidincapitalconsiderationpaidtorelatedparty,adjustmentstoadditionalpaidincapitalsharebasedcompensationrequisiteserviceperiodrecognitionvalue,allocatedsharebasedcompensationexpense,...,revenuebycountryexceeded,revenuebycountryexceedednumberofcountries,numberofmajorproductlines,longtermdebtmaturitiesrepaymentsofprincipalinnexttwelvemonths,longtermdebtmaturityafteryearfour,restructuringreservesettledwithoutcash2,contractwithcustomerliabilityrevenuerecognized,paymentstoacquireinvestments,ticker,date
0,0001555280-13-000020,2.970000e+08,8.805000e+08,9.600000e+07,9.700000e+08,-139000000.0,4.060000e+08,-3.449000e+09,8.500000e+06,8.500000e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS,2013-05-15
1,0001193125-13-230781,2.693333e+08,8.773333e+08,1.046667e+08,9.486667e+08,-72400000.0,4.060000e+08,-3.449000e+09,1.100000e+07,1.600000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS,2013-05-22
2,0001555280-13-000042,4.530000e+08,1.018500e+09,9.850000e+07,9.925000e+08,-170000000.0,4.345000e+08,-3.551000e+09,2.150000e+07,1.725000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS,2013-08-14
3,0001193125-13-366759,3.733333e+08,9.693333e+08,1.063333e+08,9.636667e+08,-84800000.0,4.345000e+08,-3.551000e+09,2.120000e+07,1.885714e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS,2013-09-13
4,0001555280-13-000114,4.005000e+08,1.005000e+09,1.000000e+08,1.003500e+09,-201000000.0,4.380000e+08,-3.551000e+09,2.750000e+07,1.675000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS,2013-11-13
5,0001555280-14-000164,4.125000e+08,1.019000e+09,1.040000e+08,1.019500e+09,-128750000.0,4.390000e+08,-3.551000e+09,3.000000e+07,3.000000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS,2014-03-26
6,0001555280-14-000213,4.345000e+08,1.119000e+09,5.000000e+07,1.047000e+09,-224000000.0,8.920000e+08,-3.449000e+09,8.000000e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS,2014-05-13
7,0001555280-14-000274,4.215000e+08,1.118000e+09,5.150000e+07,1.071000e+09,-209000000.0,9.005000e+08,-3.551000e+09,2.200000e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS,2014-08-12
8,0001555280-14-000335,3.825000e+08,1.097500e+09,5.300000e+07,1.085500e+09,-229500000.0,9.080000e+08,-3.551000e+09,3.000000e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS,2014-11-10
9,0001555280-15-000057,3.980000e+08,1.059000e+09,5.300000e+07,1.086500e+09,-200500000.0,9.180000e+08,-3.551000e+09,3.400000e+07,3.433333e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZTS,2015-02-27


In [22]:
sec.connect()
market.connect()
filings = []
year = 2021
quarter = 4
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filing = sec.retrieve_filing_data(cik)
        filing["ticker"] = ticker
        filing["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in filing["filed"]]
        filing = p.column_date_processing(filing)
        filing["quarter"] = [x.quarter for x in filing["date"]]
        filing["year"] = [x.year for x in filing["date"]]
        filings.append(filing[(filing["year"]==year) & (filing["quarter"]==quarter)])
    except Exception as e:
        print(ticker, str(e))
sec.disconnect()
market.disconnect()

 39%|█████████████████████████████▋                                              | 197/505 [01:04<02:17,  2.23it/s]

FRC 'filed'


 83%|██████████████████████████████████████████████████████████████▊             | 417/505 [02:13<00:36,  2.39it/s]

SBNY 'filed'


100%|████████████████████████████████████████████████████████████████████████████| 505/505 [02:40<00:00,  3.14it/s]


In [43]:
fudge = []
for stuff in filings:
    try:
        fudge.append(stuff[centralized].iloc[0])
    except:
        continue
financials = pd.DataFrame(fudge)
financials

,assets,liabilitiesandstockholdersequity,netcashprovidedbyusedinfinancingactivities,netcashprovidedbyusedininvestingactivities,date,ticker
60,4.781100e+10,4.781100e+10,-3.615500e+09,-8.765000e+08,2021-10-26,MMM
52,7.317150e+10,7.317150e+10,-2.800000e+09,-1.410500e+09,2021-11-03,ABT
35,1.497115e+11,1.497115e+11,-1.025350e+10,-1.906450e+10,2021-11-02,ABBV
41,1.512196e+09,1.512196e+09,-7.748500e+06,-9.485450e+07,2021-10-28,ABMD
49,4.342529e+10,4.342529e+10,-1.034456e+09,-1.179494e+09,2021-12-16,ACN
...,...,...,...,...,...,...
50,6.135500e+09,6.135500e+09,-4.285000e+08,-1.960000e+08,2021-11-03,YUM
48,5.620000e+09,5.620000e+09,-1.100000e+08,-4.960000e+08,2021-11-02,ZBRA
52,2.421315e+10,2.421315e+10,-3.928500e+08,-3.276500e+08,2021-11-04,ZBH
45,8.489250e+10,8.489250e+10,7.334000e+09,-7.877000e+09,2021-11-04,ZION


In [44]:
financials = p.column_date_processing(financials)
market.store("unified_financials",financials[centralized])

In [45]:
market.connect()
data = market.retrieve("unified_financials")
market.disconnect()

In [46]:
data["year"] = [x.year for x in data["date"]]
data["quarter"] = [x.quarter for x in data["date"]]

In [47]:
data

,assets,liabilitiesandstockholdersequity,netcashprovidedbyusedinfinancingactivities,netcashprovidedbyusedininvestingactivities,date,ticker,year,quarter
0,2.594400e+10,2.594400e+10,-1.122000e+09,-9.210000e+08,2009-07-31,MMM,2009,3
1,2.594400e+10,2.594400e+10,-1.122000e+09,-9.210000e+08,2009-07-31,MMM,2009,3
2,2.652150e+10,2.652150e+10,-2.099333e+09,-1.832667e+09,2010-02-16,MMM,2010,1
3,2.652150e+10,2.652150e+10,-2.099333e+09,-1.832667e+09,2010-02-16,MMM,2010,1
4,2.763600e+10,2.763600e+10,-5.910000e+08,-5.025000e+08,2010-05-05,MMM,2010,2
...,...,...,...,...,...,...,...,...
23003,6.135500e+09,6.135500e+09,-4.285000e+08,-1.960000e+08,2021-11-03,YUM,2021,4
23004,5.620000e+09,5.620000e+09,-1.100000e+08,-4.960000e+08,2021-11-02,ZBRA,2021,4
23005,2.421315e+10,2.421315e+10,-3.928500e+08,-3.276500e+08,2021-11-04,ZBH,2021,4
23006,8.489250e+10,8.489250e+10,7.334000e+09,-7.877000e+09,2021-11-04,ZION,2021,4


In [56]:
category_stuff = sp5[["Symbol","GICS Sector"]].rename(columns={"Symbol":"ticker"})
market.connect()
whut = []
for symbol in category_stuff["ticker"]:
    try:
        ticker_prices = market.retrieve_ticker_prices("prices",symbol)
        ticker_prices = p.column_date_processing(ticker_prices)
        ticker_prices["quarter"] = [x.quarter for x in ticker_prices["date"]]
        ticker_prices["year"] = [x.year for x in ticker_prices["date"]]
        whut.append(ticker_prices.groupby(["year","quarter","ticker"]).mean())
    except:
        continue
market.disconnect()

In [60]:
additional_categories = pd.concat(whut).reset_index().merge(category_stuff,on="ticker",how="left")
additional_categories["1_classification"] = "stock"
additional_categories["2_classification"] = additional_categories["GICS Sector"]
price_points = []
for price in additional_categories["adjclose"]:
    if price < 100:
        price_points.append(str(100))
    else:
        if price > 100 and price < 200:
            price_points.append(str(200))
        else:
            price_points.append(str(500))
additional_categories["price_point"] = price_points
additional_categories["3_classification"] = additional_categories["price_point"] + additional_categories["GICS Sector"]

In [68]:
start = data["year"].min()
end = data["year"].max()
training_years = 4
product.db.connect()
the_cats = []
for year in tqdm(range(2014,2022)):
    try:
        training_set = data[(data["year"] < year) & (data["year"] > year - training_years)].copy().dropna()
        prediction_set = data[(data["year"]==year)].copy().dropna()
        non_relevant = ["date","quarter","ticker","year","training_years"]
        factors = [x for x in training_set.columns if x not in non_relevant]
        kmeans = KMeans(n_clusters=10,random_state=42).fit(training_set[factors])
        cats = kmeans.predict(prediction_set[factors])
        prediction_set["4_classification"] = cats
        prediction_set = prediction_set.groupby(["year","quarter","ticker"]).mean().reset_index()
        prediction_set = prediction_set.merge(additional_categories[["year","quarter","ticker","1_classification","2_classification","3_classification"]]
                             ,on=["year","quarter","ticker"],how="left")
        the_cats.append(prediction_set[['year','quarter','ticker',"1_classification","2_classification","3_classification",'4_classification']].dropna())
    except Exception as e:
        print(str(e))
product.db.disconnect()

100%|████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  7.56it/s]


In [69]:
pd.concat(the_cats)

,year,quarter,ticker,1_classification,2_classification,3_classification,4_classification
0,2014,1,A,stock,Health Care,100Health Care,0.0
1,2014,1,AAL,stock,Industrials,100Industrials,6.0
2,2014,1,AAP,stock,Consumer Discretionary,200Consumer Discretionary,0.0
3,2014,1,AAPL,stock,Information Technology,100Information Technology,3.0
4,2014,1,ABC,stock,Health Care,100Health Care,0.0
...,...,...,...,...,...,...,...
1810,2021,4,XLNX,stock,Information Technology,200Information Technology,0.0
1811,2021,4,XOM,stock,Energy,100Energy,8.0
1812,2021,4,XRAY,stock,Health Care,100Health Care,0.0
1813,2021,4,XYL,stock,Industrials,200Industrials,0.0


In [70]:
stuff = pd.concat(the_cats)
product.db.connect()
sim = product.db.store("sim",stuff)
product.db.disconnect()